About the dataset : 
1. id     : unique id for a news article
2. tittle : the title of a news artickle
3. author : author of the news artickle
4. text   : the tect of the article, could be incomplete
5. label  : a label that maks whether the news article is real or fake

   1 : fake
   0 : real

Tentang dataset:
1. id: id unik untuk sebuah artikel berita
2. judul: judul dari sebuah artikel berita
3. penulis: penulis dari artikel berita
4. teks: teks dari artikel, bisa jadi tidak lengkap
5. label: label yang menandakan apakah artikel berita itu nyata atau palsu
   1 : fake
   0 : real

In [2]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


False

In [4]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Preprocessing

In [5]:
# loading the dataset
df = pd.read_csv('/kaggle/input/fake-news/train.csv')
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [6]:
df.shape

(20800, 5)

In [7]:
# counting the number of missing values
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
# replacing the nun values with empty string
df = df.fillna('')

In [9]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [10]:
# marge author name + title
df['content'] = df['author'] + ' ' + df['title']

In [11]:
df.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [12]:
# separating the data and label
X = df.drop(columns= 'label', axis=1)
y = df['label']

In [13]:
X

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...","Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [14]:
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

Steening : 

Steening is the process of reducing convert to tirs root word.

Example : 
actor, actrees, actiing  -- > act

In [15]:
port_stem = PorterStemmer()

In [16]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [17]:
#test
stemming('Lugastyan hai i love you so Much +)+((+')

'lugastyan hai love much'

In [18]:
df['content'] = df['content'].apply(stemming)

In [19]:
df.sample(5)

,id,title,author,text,label,content
4156,4156,2017 N.C.A.A. Tournament Bracket: Who Has the ...,"Marc Tracy, Zach Schonbrun and Victor Mather",Villanova ( ) which won last year’s N. C. A. A...,0,marc traci zach schonbrun victor mather n c to...
303,303,"After Berkeley, Treat the Violent, Anti-Speech...",Joel B. Pollak,The violence that stopped Breitbart tech edito...,0,joel b pollak berkeley treat violent anti spee...
9552,9552,Penguins Finish Off Sharks to Win Stanley Cup ...,David Pollak,"SAN JOSE, Calif. — Much of Game 6 of the St...",0,david pollak penguin finish shark win stanley ...
11514,11514,Voting Machine SHOCKER: Video Proof Election I...,Baxter Dmitry,"Posted on October 28, 2016 by Baxter Dmitry in...",1,baxter dmitri vote machin shocker video proof ...
983,983,Detained Illegal Aliens End 3-Day Hunger Strike,Bob Price,Illegal immigrants detained at the Northwest D...,0,bob price detain illeg alien end day hunger st...


In [20]:
# separating the data and label
X = df['content'].values
y = df['label'].values 

In [21]:
X.shape, y.shape

((20800,), (20800,))

In [22]:
X

array(['darrel lucu hous dem aid even see comey letter jason chaffetz tweet',
       'daniel j flynn flynn hillari clinton big woman campu breitbart',
       'consortiumnew com truth might get fire', ...,
       'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time',
       'alex ansari nato russia hold parallel exercis balkan',
       'david swanson keep f aliv'], dtype=object)

In [23]:
y

array([1, 0, 1, ..., 0, 1, 1])

In [24]:
# converting content with TFIDF

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)



In [25]:
X

<20800x17141 sparse matrix of type '<class 'numpy.float64'>'
	with 210632 stored elements in Compressed Sparse Row format>

In [26]:
print(X)

  (0, 15697)	0.28485063562728646
  (0, 2485)	0.3676519686797209
  (0, 7698)	0.24785219520671603
  (0, 8636)	0.29212514087043684
  (0, 2962)	0.2468450128533713
  (0, 13480)	0.2565896679337957
  (0, 4979)	0.233316966909351
  (0, 268)	0.27010124977708766
  (0, 3795)	0.2705332480845492
  (0, 7012)	0.21874169089359144
  (0, 8915)	0.3635963806326075
  (0, 3603)	0.3598939188262559
  (1, 1895)	0.15521974226349364
  (1, 2224)	0.3827320386859759
  (1, 16812)	0.30071745655510157
  (1, 1498)	0.2939891562094648
  (1, 2816)	0.19094574062359204
  (1, 6823)	0.1904660198296849
  (1, 5510)	0.7143299355715573
  (1, 3571)	0.26373768806048464
  (2, 5396)	0.3866530551182615
  (2, 5975)	0.3474613386728292
  (2, 9625)	0.49351492943649944
  (2, 15622)	0.41544962664721613
  (2, 2946)	0.3179886800654691
  :	:
  (20797, 1288)	0.3353805680413986
  (20797, 13128)	0.24825263521976057
  (20797, 12350)	0.27263457663336677
  (20797, 14976)	0.3115945315488075
  (20797, 12144)	0.24778257724396505
  (20797, 9523)	0.295420

Splitting the dataset training & test

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=2)

In [28]:
X_train.shape, y_train.shape

((16640, 17141), (16640,))

In [29]:
X_test.shape, y_test.shape

((4160, 17141), (4160,))

Training model logistic regression

In [30]:
model = LogisticRegression()

In [31]:
model.fit(X_train, y_train)

LogisticRegression()

Evaluation

In [32]:
# accuracy in training data

X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, y_train)
training_data_accuracy

0.9864783653846154

In [33]:
# accuracy in test data

X_test_prediction = model.predict(X_test)
training_data_accuracy = accuracy_score(X_test_prediction, y_test)
training_data_accuracy

0.9790865384615385

Making Predictive System

In [34]:
df.loc[df['label'] == 0].index

Index([    1,     5,     7,     8,     9,    10,    11,    15,    16,    19,
       ...
       20783, 20785, 20787, 20789, 20790, 20792, 20794, 20795, 20796, 20797],
      dtype='int64', length=10387)

In [35]:
df['content'][20796]

'benjamin hoffman n f l playoff schedul matchup odd new york time'

In [36]:
df['label'][20796]

0

In [40]:
test = input()
test = [stemming(test)]
test = vectorizer.transform(test)
prediction = model.predict(test)
if (prediction[0] == 1):
    print('')
    print('the news is real')
else:
    print('')
    print('the news is fake')


 But this night was different. The mood was buoyant. Champagne was popped.  “Raise a glass to President Trump,” Micki Witthoeft, the group’s leader, told the crowd, offering a toast to the man who had become the president-elect that morning.  Witthoeft is the mother of Ashli Babbitt, a 35-year-old Air Force veteran and fervent Donald Trump supporter who was fatally shot by a police officer inside the US Capitol on January 6, 2021, as she tried to breach an area near the House floor while lawmakers fled.  The people behind the vigil are the tip of the spear of a national movement of activists who have spent years campaigning for the release of January 6 defendants. They’ve pressed forward even though a majority of Americans still see the Capitol rioters as responsible for an attack against democracy, according to the most recent polling.  Now, Witthoeft and others expect Trump to make good on his oft-repeated campaign pledge to pardon the January 6 “political prisoners,” as they are cal


the news is real


In [41]:
model.predict(test)

array([1])